In [ ]:
obj = create_inverted_index()

In [ ]:
import pickle

In [ ]:
with open('reverse_index.pickle', 'rb') as file:
    obj = pickle.load(file)

In [ ]:
docs[docs['id'].apply(lambda x : x in [3119780, 324804, 2495750, 1794250, 9386922, 9654542, 9477456, 9822035, 9120694, 7284024])]

In [ ]:
import pickle
with open('reverse_index.pickle', 'wb') as handle:
    pickle.dump(obj, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
from flask import Flask
from flask import request
import nltk.stem.porter as stemmer
import json
import requests
import numpy as np

app = Flask(__name__)

@app.route("/reverseindex", methods=["POST"])
def reverceindex():
    j = request.json
    words = j['data']
    
    documents = list(intersect_all(words, obj))
    
    if j.get('max_docs'):
        documents = documents[:j.get('max_docs')]
        print (documents)
    
    documents = docs[docs['id'].apply(lambda x : x in documents)].loc[:, ['id', 'text']].to_dict('records')
    
    return json.dumps({"status":"ok", "got_data":j['data'], "processed_data": documents})


@app.route("/reverseindex/add", methods=['POST'])
def add():
    '''
    Add doc to reverse index
    '''
    pass
    

if __name__ == "__main__":
    app.run(host='0.0.0.0', port=13538)

In [ ]:
import pandas as pd
docs = pd.read_csv('../../Data/eval_texts.csv', sep='\t')
docs.drop_duplicates(subset='id', keep='first', inplace=True)
docs.head()

In [ ]:
import pandas as pd
from collections import Counter

def position_in_text(lst, term):
    return [i for i, x in enumerate(lst) if x == term]

def create_inverted_index():   
    inverted_index = dict()
    docs = pd.read_csv('../../Data/eval_texts.csv', sep='\t')
    for i in range(docs.shape[0]):
        doc = dict()
        doc['docID'] = int(docs.loc[i, 'id'])
        #doc['text'] = docs.loc[i, 'text_searchable']
        response_analyze = requests.post('http://127.0.0.1:13533/analyze', json={'data' : docs.loc[i, 'text']})
        doc['text_searchable'] = json.loads(response_analyze.text)['words']
        
        #tokens = doc['text'].split(' ')
        tokens = doc['text_searchable']
        number_of_occurrences = Counter(tokens)         
        for term in set(tokens):#number_of_occurrences.keys():
          
          if term not in inverted_index:
            inverted_index[term] = [ {'docID': doc['docID'], 'count': number_of_occurrences[term], 'pos': position_in_text(tokens, term)}]
          
          else:
            inverted_index[term].append({'docID': doc['docID'], 'count': number_of_occurrences[term], 'pos': position_in_text(tokens, term)})
            
    return(inverted_index)

In [ ]:
def intersect_all(list_of_term, inverted_index):
    
    ans = set()
    for term in list_of_term:            
        p = inverted_index.get(term)
        posting_list=set()
        if not p:
            return set()
        for d in p:
            posting_list.add(d['docID'])
        if len(ans) == 0:
            ans = posting_list
        else:
            ans = ans & posting_list

    return ans

In [ ]:
def intersect(term1, term2, inverted_index):
    answer = []
    p1, p2 = inverted_index[term1], inverted_index[term2]
  
    posting_list1, posting_list2 = [], []
  
    for d in p1:
        posting_list1.append(d['docID'])
    for d in p2:
        posting_list2.append(d['docID'])
    
    posting_list1.sort()
    posting_list2.sort()
    
    i,j = 0, 0    
    while i < len(posting_list1) and j < len(posting_list2):
        if posting_list1[i] == posting_list1[j]:
            answer.append(posting_list1[i])
            i+=1
            j+=1
        elif posting_list1[i] < posting_list1[j]:
            i+=1
        else:
            j+=1
      
    return answer